In [1]:
import numpy as np
import glob
import subprocess
import os
import csv
import unicodedata
import re
import pandas as pd
import string
from time import sleep
import tqdm
import json
from ethnicolr import census_ln, pred_census_ln,pred_wiki_name
from ethnicolr import pred_fl_reg_name
import argparse


In [42]:
ext = "F_"
names = pd.read_csv(f"../data/{ext}names.csv")
names = names.iloc[:,1:]
names.head()

,full,first,F_prob.w,F_prob.m,last,__name,rowindex,asian_mean,asian_std,asian_lb,...,hispanic_ub,nh_black_mean,nh_black_std,nh_black_lb,nh_black_ub,nh_white_mean,nh_white_std,nh_white_lb,nh_white_ub,race
0,"KEREN, RON",RON,0.01,0.99,KEREN,Keren Ron,0,0.019782,0.022264,0.003209,...,0.003799,0.046234,0.027541,0.011829,0.013075,0.926311,0.034857,0.764413,0.810378,nh_white
1,"BUTENSKY, ADAM",ADAM,0.01,0.99,BUTENSKY,Butensky Adam,1,0.015387,0.015652,0.001880,...,0.008618,0.081553,0.047101,0.008702,0.013769,0.872467,0.058715,0.656821,0.703254,nh_white
2,"MAZZEO, PAMELA",PAMELA,0.98,0.02,MAZZEO,Mazzeo Pamela,2,0.187527,0.097439,0.022175,...,0.019488,0.126334,0.068297,0.020817,0.031844,0.627018,0.119830,0.353854,0.363949,nh_white
3,"WEINERMAN, BENNETT",BENNETT,0.01,0.99,WEINERMAN,Weinerman Bennett,3,0.013856,0.011343,0.000909,...,0.004581,0.078042,0.112157,0.004535,0.006895,0.851487,0.131649,0.200429,0.338612,nh_white
4,"WEINREB, SCOTT",SCOTT,0.00,1.00,WEINREB,Weinreb Scott,4,0.007190,0.007782,0.000828,...,0.002846,0.016715,0.014999,0.002312,0.002835,0.967110,0.021874,0.897921,0.899085,nh_white


In [43]:
for name in tqdm.tqdm(names.index): 
    try: names.loc[name,'last'] = names.loc[name,'full'].split(',')[0]
    except: names.loc[name,'last'] = ""
names.head()

100%|█████████████████████████████████████| 2061/2061 [00:00<00:00, 4808.58it/s]


,full,first,F_prob.w,F_prob.m,last,__name,rowindex,asian_mean,asian_std,asian_lb,...,hispanic_ub,nh_black_mean,nh_black_std,nh_black_lb,nh_black_ub,nh_white_mean,nh_white_std,nh_white_lb,nh_white_ub,race
0,"KEREN, RON",RON,0.01,0.99,KEREN,Keren Ron,0,0.019782,0.022264,0.003209,...,0.003799,0.046234,0.027541,0.011829,0.013075,0.926311,0.034857,0.764413,0.810378,nh_white
1,"BUTENSKY, ADAM",ADAM,0.01,0.99,BUTENSKY,Butensky Adam,1,0.015387,0.015652,0.001880,...,0.008618,0.081553,0.047101,0.008702,0.013769,0.872467,0.058715,0.656821,0.703254,nh_white
2,"MAZZEO, PAMELA",PAMELA,0.98,0.02,MAZZEO,Mazzeo Pamela,2,0.187527,0.097439,0.022175,...,0.019488,0.126334,0.068297,0.020817,0.031844,0.627018,0.119830,0.353854,0.363949,nh_white
3,"WEINERMAN, BENNETT",BENNETT,0.01,0.99,WEINERMAN,Weinerman Bennett,3,0.013856,0.011343,0.000909,...,0.004581,0.078042,0.112157,0.004535,0.006895,0.851487,0.131649,0.200429,0.338612,nh_white
4,"WEINREB, SCOTT",SCOTT,0.00,1.00,WEINREB,Weinreb Scott,4,0.007190,0.007782,0.000828,...,0.002846,0.016715,0.014999,0.002312,0.002835,0.967110,0.021874,0.897921,0.899085,nh_white


In [44]:
race = pred_fl_reg_name(names,'last','first')

['asian', 'hispanic', 'nh_black', 'nh_white']


In [45]:
prefixes = [(n,ext + n) for n in names.columns[7:]]
names.rename(columns = dict(prefixes), inplace=True)
names.head()

,full,first,F_prob.w,F_prob.m,last,__name,rowindex,F_asian_mean,F_asian_std,F_asian_lb,...,F_hispanic_ub,F_nh_black_mean,F_nh_black_std,F_nh_black_lb,F_nh_black_ub,F_nh_white_mean,F_nh_white_std,F_nh_white_lb,F_nh_white_ub,F_race
0,"KEREN, RON",RON,0.01,0.99,KEREN,Keren Ron,0,0.019782,0.022264,0.003209,...,0.003799,0.046234,0.027541,0.011829,0.013075,0.926311,0.034857,0.764413,0.810378,nh_white
1,"BUTENSKY, ADAM",ADAM,0.01,0.99,BUTENSKY,Butensky Adam,1,0.015387,0.015652,0.001880,...,0.008618,0.081553,0.047101,0.008702,0.013769,0.872467,0.058715,0.656821,0.703254,nh_white
2,"MAZZEO, PAMELA",PAMELA,0.98,0.02,MAZZEO,Mazzeo Pamela,2,0.187527,0.097439,0.022175,...,0.019488,0.126334,0.068297,0.020817,0.031844,0.627018,0.119830,0.353854,0.363949,nh_white
3,"WEINERMAN, BENNETT",BENNETT,0.01,0.99,WEINERMAN,Weinerman Bennett,3,0.013856,0.011343,0.000909,...,0.004581,0.078042,0.112157,0.004535,0.006895,0.851487,0.131649,0.200429,0.338612,nh_white
4,"WEINREB, SCOTT",SCOTT,0.00,1.00,WEINREB,Weinreb Scott,4,0.007190,0.007782,0.000828,...,0.002846,0.016715,0.014999,0.002312,0.002835,0.967110,0.021874,0.897921,0.899085,nh_white


In [46]:
names.to_csv(f"../data/{ext}names.csv")